# A tutorial on fitting exchange parameters to DFT energies<br/>The Case of Iron

James K. Glasbrenner

November 28, 2017

In [ ]:
# SETUP BLOCK
%matplotlib inline
import pandas as pd
import pymatgen as pmg
import numpy as np
from pymatgen.transformations.standard_transformations import (
    RotationTransformation, SupercellTransformation)
from pymatgen.analysis.structure_analyzer import OrderParameters
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score

# R
from rpy2.robjects import pandas2ri, default_converter, StrVector
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.lib.tidyr import DataFrame as tibble
from rpy2.robjects.lib.dplyr import (mutate,
                                     group_by,
                                     summarize,
                                     select,
                                     left_join)
from rpy2.robjects.lib.dplyr import filter as rfilter

## Theoretical background

## Generate the crystal structure using pymatgen

At room temperature, [iron has the following crystal structure][iron-crystal]:

*   Structure: BCC
*   Space group: Im-3m
*   Space group number: 229
*   Lattice constants
    *   $a = 2.8665 \text{ Å}$.
    
This is a straightforward crystal structure and can easily be constructed by hand.
However, we will make use of the Python library `pymatgen`, which allows us to construct and manipulate crystal structures in a systematic and reproducible way.

<!-- Implicit links -->

[iron-crystal]: https://www.webelements.com/iron/crystal_structure.html

In [ ]:
iron = {
    "spacegroup": 229,
    "lattice": {
        "lengths": (2.8665, 2.8665, 2.8665),
        "angles": (90, 90, 90)
    },
    "structure": {
        "species": ["Fe"],
        "basis_coordinates": [[0.00, 0.00, 0.00]]
    },
    "pymatgen": {
        "lattice": None,
        "structure": None
    }
}

In [ ]:
iron["pymatgen"]["lattice"] = pmg.Lattice.from_lengths_and_angles(
    abc=iron["lattice"]["lengths"],
    ang=iron["lattice"]["angles"]
)

iron["pymatgen"]["structure"] = pmg.Structure.from_spacegroup(
    sg=iron["spacegroup"],
    lattice=iron["pymatgen"]["lattice"],
    species=iron["structure"]["species"],
    coords=iron["structure"]["basis_coordinates"]
)

To construct the spin Hamiltonian, we need to sub-divide the crystal into sub-lattices.
In a supercell with a cubic lattice, each basis atom corresponds to a separate sub-lattice (note that some sublattices may be equivalent due to symmetry).
The default cubic cell that `pymatgen` has given us is as follows:

In [ ]:
fe = iron["pymatgen"]["structure"]
fe

There are two basis atoms, one located at `(0, 0, 0)` and the other at `(0.5, 0.5, 0.5)`.
We will call these sublattices 1 and 2.
We use the `add_site_property` method to embed this label into the structure object:

In [ ]:
fe.add_site_property("sublattice", [1, 2])

Next, we want to find the intra- and inter-sublattice neighbors for each site up to a given distance.
Within the intra- and inter-sublattice groups, we will additionally group by distance.
For our model, interactions between sites the same distance apart within an intra- or inter-sublattice grouping are considered to be equivalent.
Thus, we only need to count how many of these equivalent neighbors we have, and afterwards we can start constructing the coefficients within our Heisenberg model.

We use the neighbor finding method in `pymatgen` to grab the relevant neighbors:

In [ ]:
neighbors = fe.get_all_neighbors(r=4.5, include_index=True)

For convenience, we convert the information in `neighbors` into a `pandas` data frame:

In [ ]:
neighbor_distances = []
for site_i, site_neighbors in enumerate(neighbors):
    sublattice_i = fe[site_i].properties["sublattice"]
    neighbor_distances.extend([
       [site_i, site_j[2], sublattice_i, fe[site_j[2]].properties["sublattice"], site_j[1]]
       for site_j in site_neighbors
   ])
neighbor_distances = pd.DataFrame(
    data=neighbor_distances, columns=("i", "j", "sublattice_i", "sublattice_j", "distance"))

We then use the `groupby` method to help us find the unique distances within intra- and inter-sublattice groups and count the number of sites that fall within a grouping.
After iterating over all the basis sites, we then divide the counts by the number of atoms in the cell in order to normalize our result.

In [ ]:
grouped_neighbors = neighbor_distances.round(decimals=7).groupby(
    ["sublattice_i", "sublattice_j", "distance"])
nn_list = []
for group in grouped_neighbors:
    group[1]["sublattice_i"].unique()
    nn_list.append(
        [group[0][0],
         group[0][1],
         group[0][2],
         np.int(group[1]["distance"].count() / len(neighbors))
        ]
    )

neighbor_counts = pd.DataFrame(data=nn_list, columns=("sublattice_i", "sublattice_j", "distance", "count")).sort_values(["distance"])
neighbor_counts

Next, let's associate the different exchange parameters with the different sublattice combinations:

In [ ]:
neighbor_counts["exchange"] = ["J1", "J1", "J2", "J2", "J3", "J3"]

Define the magnetic patterns by specifying the spin directions on each sublattice and labelling it:

In [ ]:
magnetic_states = pd.DataFrame(data={
    "sublattice_i": [1, 1, 2, 2, 1, 1, 2, 2],
    "sublattice_j": [1, 2, 1, 2, 1, 2, 1, 2],
    "spin_i": [1, 1, 1, 1, 1, 1, -1, -1],
    "spin_j": [1, 1, 1, 1, 1, -1, 1, -1],
    "pattern": ["FM", "FM", "FM", "FM", "AFM", "AFM", "AFM", "AFM"]
})

Use the `rpy2` package to call up an `R` instance for data manipulation (I know how to use `dplyr` better than `pandas`), and use all our data frames to compute the Hamiltonian coefficients:

In [ ]:
with localconverter(default_converter + pandas2ri.converter) as cv:
    pairwise_interaction = (
        tibble(magnetic_states).
        mutate(interaction = "-(spin_i * spin_j)").
        select("sublattice_i", "sublattice_j", "pattern", "interaction"))

    hamiltonian_coefficients = (
        tibble(neighbor_counts).
        left_join(pairwise_interaction, by = StrVector(["sublattice_i", "sublattice_j"])).
        group_by("pattern", "exchange").
        summarize(coeff = "sum(count * interaction)").
        arrange("desc(pattern)"))

pairwise_interaction = pandas2ri.converter.ri2py(pairwise_interaction)
hamiltonian_coefficients = pandas2ri.converter.ri2py(hamiltonian_coefficients)

The table of coefficients is therefore:

In [ ]:
hamiltonian_coefficients

The DFT energies we have are:

In [ ]:
dft_nm = -2569.13251613 * pmg.units.Ha_to_eV / 2
dft = pd.DataFrame(data={
    "energy": [
        -2569.17660341 * pmg.units.Ha_to_eV / 2 - dft_nm,
        -2569.14233549 * pmg.units.Ha_to_eV / 2 - dft_nm
    ],
    "pattern": ["FM", "AFM"]})
dft

Preprocess the dataframe to suit linear regression:

In [ ]:
final_df = hamiltonian_coefficients.join(dft.set_index("pattern"), on="pattern")
with localconverter(default_converter + pandas2ri.converter) as cv:
    final_df = (tibble(final_df).
                spread(key = "exchange", value = "coeff").
                select("J1", "J2", "energy"))

final_df = pandas2ri.ri2py(final_df)
explanatory = final_df[["J1"]]
response = final_df["energy"]

Run the fitting procedure:

In [ ]:
# Create linear regression object
lm = linear_model.LinearRegression()

# Train the model using the training sets
lm.fit(explanatory, response)

# Make predictions using the testing set
energy_pred = lm.predict(explanatory) * 1000
print(energy_pred, lm.coef_ * 1000, lm.intercept_ * 1000)